In [92]:
# Load libraries
library(tidyverse)

In [93]:
# This cell downloads the original data from data.bs.ch and saves it in the raw folder
# It has been downloaded on 2024-03-01

# download_link <- "https://data.bs.ch/api/explore/v2.1/catalog/datasets/100138/exports/csv?lang=de&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
# download.file(download_link, destfile = "/raw/100138.csv")

In [94]:
# Load original data
original_data <- read_delim("raw/100138.csv", delim = ";")

Rows: 1664159 Columns: 23
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr  (14): Wanderungstyp, Wochentag, Staatsangehörigkeit, Geschlecht, Von Ko...
dbl   (7): Jahr, Monat, Kalenderwoche, Tag-Nr., Alter, Aufenthaltsdauer in J...
date  (2): Datum, Startdatum Woche

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [95]:
# Peek into the original data
head(original_data)

Wanderungstyp,Datum,Jahr,Monat,Kalenderwoche,Startdatum Woche,Tag-Nr.,Wochentag,Staatsangehörigkeit,Geschlecht,⋯,Von Land,Von Kanton,Von Gemeinde,Von Wohnviertel,Nach Kontinent,Nach Land,Nach Kanton,Nach Gemeinde,Nach Wohnviertel,Anzahl
<chr>,<date>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Zuzug,2003-08-08,2003,8,32,2003-08-04,220,Fr,Ausländer,W,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Riehen,Riehen,1
Zuzug,2003-08-08,2003,8,32,2003-08-04,220,Fr,Ausländer,W,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Riehen,Riehen,1
Zuzug,2003-08-07,2003,8,32,2003-08-04,219,Do,Ausländer,M,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Basel,Breite,1
Zuzug,2003-08-07,2003,8,32,2003-08-04,219,Do,Ausländer,W,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Basel,Gundeldingen,1
Zuzug,2003-08-07,2003,8,32,2003-08-04,219,Do,Schweizer,M,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Basel,St. Alban,2
Zuzug,2003-08-07,2003,8,32,2003-08-04,219,Do,Schweizer,W,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Basel,Iselin,1


In [96]:
data <- filter(original_data, Wanderungstyp != "Umzug")

In [97]:
# Save new dataframe with less columns
data <- select(data,
  -"Datum",
  -"Kalenderwoche",
  -"Startdatum Woche",
  -"Tag-Nr.",
  -"Wochentag",
  -"Aufenthaltsdauer in Jahren",
  -"Von Gemeinde",
  -"Nach Gemeinde"
)

In [98]:
# Generate factors
columns <- c(
  "Wanderungstyp",
  "Staatsangehörigkeit",
  "Geschlecht",
  "Von Kontinent",
  "Von Land",
  "Von Kanton",
  "Von Wohnviertel",
  "Nach Kontinent",
  "Nach Land",
  "Nach Kanton",
  "Nach Wohnviertel"
)
data[, columns] <- lapply(data[, columns], as.factor)

In [99]:
# Remove spaces from column names
colnames(data) <- make.names(colnames(data))

In [100]:
# Rename column names
data <- rename(data,
  Staatsangehoerigkeit = Staatsangehörigkeit,
  VonKontinent = Von.Kontinent,
  VonLand = Von.Land,
  VonKanton = Von.Kanton,
  VonWohnviertel = Von.Wohnviertel,
  NachKontinent = Nach.Kontinent,
  NachLand = Nach.Land,
  NachKanton = Nach.Kanton,
  NachWohnviertel = Nach.Wohnviertel
)

In [101]:
# Sort data
data <- arrange(data, Jahr, Monat)

In [102]:
# Repeat each row n times if Anzahl is > 1
# For the final visualization, we want to have an individual data point for each migration

# Create a vector of row indices to repeat
row_indices <- rep(seq_len(nrow(data)), ifelse(data$Anzahl > 1, data$Anzahl - 1, 0))

# Create the expanded dataframe by indexing the original dataframe
expanded_data <- data[row_indices, ]

In [105]:
# Bind the copied rows to the project data
data <- rbind(data, expanded_data)

In [106]:
# Re-sort the project data
data <- arrange(data, Jahr, Monat, Alter)

In [107]:
# Remove the Anzahl column, it's no longer needed
data <- select(data, -"Anzahl")

In [108]:
# Peek into the project data
head(data)
tail(data)
summary(data)

Wanderungstyp,Jahr,Monat,Staatsangehoerigkeit,Geschlecht,Alter,VonKontinent,VonLand,VonKanton,VonWohnviertel,NachKontinent,NachLand,NachKanton,NachWohnviertel
<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
Wegzug,1985,1,Ausländer,M,0,Schweiz,Schweiz,BS,St. Alban,unbekannt,NA,NA,NA
Zuzug,1985,1,Schweizer,M,1,Schweiz,NA,NA,NA,Schweiz,Schweiz,BS,Riehen
Zuzug,1985,1,Schweizer,W,1,Schweiz,NA,NA,NA,Schweiz,Schweiz,BS,St. Johann
Zuzug,1985,1,Ausländer,W,1,Europa (ohne Schweiz),NA,NA,NA,Schweiz,Schweiz,BS,Vorstädte
Zuzug,1985,1,Ausländer,M,1,Europa (ohne Schweiz),NA,NA,NA,Schweiz,Schweiz,BS,Gundeldingen
Zuzug,1985,1,Ausländer,M,1,Europa (ohne Schweiz),NA,NA,NA,Schweiz,Schweiz,BS,St. Alban


Wanderungstyp,Jahr,Monat,Staatsangehoerigkeit,Geschlecht,Alter,VonKontinent,VonLand,VonKanton,VonWohnviertel,NachKontinent,NachLand,NachKanton,NachWohnviertel
<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
Wegzug,2023,9,Ausländer,M,81,Schweiz,Schweiz,BS,Matthäus,Schweiz,Schweiz,BL,NA
Zuzug,2023,9,Schweizer,M,82,Schweiz,Schweiz,BL,NA,Schweiz,Schweiz,BS,Riehen
Wegzug,2023,9,Ausländer,W,86,Schweiz,Schweiz,BS,Vorstädte,unbekannt,unbekannt,unbekannt,NA
Zuzug,2023,9,Schweizer,W,86,Unbekannt,Unbekannt,Unbekannt,NA,Schweiz,Schweiz,BS,Hirzbrunnen
Zuzug,2023,9,Schweizer,W,87,Schweiz,Schweiz,BL,NA,Schweiz,Schweiz,BS,St. Johann
Wegzug,2023,9,Schweizer,W,97,Schweiz,Schweiz,BS,Gundeldingen,Schweiz,Schweiz,VS,NA


 Wanderungstyp        Jahr          Monat        Staatsangehoerigkeit
 Wegzug:476652   Min.   :1985   Min.   : 1.000   Ausländer:525898    
 Zuzug :503397   1st Qu.:1994   1st Qu.: 4.000   Schweizer:454151    
                 Median :2005   Median : 7.000                       
                 Mean   :2004   Mean   : 6.667                       
                 3rd Qu.:2014   3rd Qu.:10.000                       
                 Max.   :2023   Max.   :12.000                       
                                                                     
 Geschlecht     Alter                           VonKontinent   
 M:526448   Min.   :  0.00   Schweiz                  :717745  
 W:453601   1st Qu.: 23.00   Europa (ohne Schweiz)    :197247  
            Median : 29.00   Asien                    : 18002  
            Mean   : 30.73   Unbekannt                : 17432  
            3rd Qu.: 37.00   Nordamerika              : 16821  
            Max.   :138.00   Lateinamerika und Karibik: 

In [109]:
# Save project data as CSV
write_csv(data, "tidy/migration.csv")

In [110]:
# In order to host the data on GitHub, we need to divide the migration data into smaller chunks.
# GitHub has a file size limit of 50MB.
# However, we are going to create much more chunks - one per month!!
# On the website, users will be able to select a starting year and month.
# The site will load only the needed data.
# Once the animation is done, the subsequent data will be loaded.

In [111]:
# Read final data
final_data <- read_delim("tidy/migration.csv", delim = ",")

Rows: 980049 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (11): Wanderungstyp, Staatsangehoerigkeit, Geschlecht, VonKontinent, Von...
dbl  (3): Jahr, Monat, Alter

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [112]:
chunks <- final_data %>%
  mutate(Jahr_Monat = paste(Jahr, Monat, sep = "-")) %>%
  group_by(Jahr_Monat) %>%
  group_split()

In [113]:
# Approach of creating chunks of data based on number of rows
# chunk_size <- 300000
# chunks <- split(final_data, (seq_len(nrow(final_data)) - 1) %/% chunk_size)

In [114]:
# Save each chunk as a separate csv file
output_dir <- "tidy/chunks/"
for (i in seq_along(chunks)) {
  write_csv(chunks[[i]], paste0(output_dir, chunks[[i]][1, "Jahr_Monat"], ".csv"))
}